In [17]:
# Importing Libraries
import json, os, requests, copy
import pandas as pd
from tkinter import *

In [18]:
# Fetching API Key from Environment Variables
api_key = os.environ.get("YT_apikey")

In [19]:
def get_user_response():
    """
    Creates a window to record user responses to the Channel Name And/Or
    Channel ID they wish to know about
    """

    root = Tk()
    root.title("Form")
    root.geometry("600x300")

    # Initialising variables to store the Channel Name
    # And/Or Channel ID input by the User
    c_name = StringVar(root)
    c_id = StringVar(root)

    def submit():
        """
        States the function of the Submit Button, i.e
        to store the responses in variables and close
        the window upon a valid response
        """
        
        nonlocal c_name
        c_name = channel_name_entry.get()

        nonlocal c_id
        c_id = channel_id_entry.get()
        
        nonlocal root
        root.destroy()

    # Creating Labels
    channel_name_label = Label(root, text="Channel User Name")
    channel_id_label = Label(root, text="Channel ID")

    # Creating Entry Boxes
    channel_name_entry = Entry(root)
    channel_id_entry = Entry(root)

    # Creating a Submit Button
    sub_button = Button(root, text="Submit", command=submit)

    # Positioning Submit, Label and Entry Boxes
    channel_name_label.grid(row=0, column=0)
    channel_id_label.grid(row=1, column=0)
    channel_name_entry.grid(row=0, column=1)
    channel_id_entry.grid(row=1, column=1)
    sub_button.grid(row=4, column=1)

    root.mainloop()

    return c_name, c_id

In [38]:
def get_channel_stats(channel_name="", channel_id=""):
    """
    Returns some basic statistics about the channel (Title, Description, Views, Subscribers, Number of Videos) and the Upload ID
    associated with that channel, which will be used to collect info about all the videos of that Channel
    
    Required Arguements (Any 1 or Both):
    channel_name:   User Name of the YouTube Channel (Sometimes the Display Name and the Actual User Name of the Channel may         differ)
    channel_id:     Unique Channel ID associated with a YouTube Channel
    """
    
    # Initialise the tkinter window to store User response
    channel_name, channel_id = get_user_response()
    
    base_channel_url = "https://www.googleapis.com/youtube/v3/channels"
    
    if channel_id != "":
        params_dict = {
            "key": api_key,
            "id": channel_id,
            "part": "contentDetails, statistics, snippet"
        }
    else:
        params_dict = {
            "key": api_key,
            "forUsername": channel_name,
            "part": "contentDetails, statistics, snippet"
        }

    response_obj = requests.get(base_channel_url, params=params_dict)
    response = json.loads(response_obj.text)
    
    try:
        resp_items = response["items"]

        channel_uid = resp_items[0]["contentDetails"]["relatedPlaylists"]["uploads"]
        channel_name = resp_items[0]["snippet"]["title"]
        channel_desc = resp_items[0]["snippet"]["description"]
        views = int(resp_items[0]["statistics"]["viewCount"])
        subs = int(resp_items[0]["statistics"]["subscriberCount"])
        num_videos = int(resp_items[0]["statistics"]["videoCount"])
    
        return (channel_uid, channel_name, channel_desc, views, subs, num_videos)
    
    except:
        print("""Please enter a valid Channel Name And/Or Channel ID!
        
Note: It is better to input a Channel ID instead of Channel Name as it is easier
to find the correct Channel by it's Unique ID rather than it's Username, which
might be different from it's Display Name
""")
        return get_channel_stats()

In [39]:
# Storing Channel Statitics in Variables
uid, title, desc, views, subs, vid_count = get_channel_stats()
# ml_id = "UC8CmTnoR19lElKS5rOIk84g"

# Formatting Numbers to inclue commas for better readability
views = f"{views:,}"
subs = f"{subs:,}"
vid_count = f"{vid_count:,}"

In [24]:
def get_vid_ids(aList, upload_id):
    """
    Returns a List of all the video IDs associated with that Channel

    Required Arguements:
    aList:          A List where to store all the video IDs
    upload_id:      The Unique Upload ID of the channel fetched from the previous function (get_channel_stats)

    """
    
    base_playlist_url = "https://www.googleapis.com/youtube/v3/playlistItems"
    params_dict = {
        "part": "snippet",
        "playlistId": upload_id,
        "maxResults": 50,
        "key": api_key
    }
    
        
    response_obj = requests.get(base_playlist_url, params=params_dict)
    response = json.loads(response_obj.text)
    resp_items = response["items"]
    
    for item in resp_items:
        aList.append(item["snippet"]["resourceId"]["videoId"])
        
    """
    Since the API can only fetch 50 results per query, it will generate a "nextPageToken" specifying the 
    place to look at for the next 50 results. It will continue to do so until the results left to query 
    are < 50. If the Channel has < 50 videos, it will not generate the "nextPageToken".

    This block checks if there is a "nextPageToken" returned with the query. If so, it executes a while
    loop till the query keeps returning a "nextPageToken". It stops when the token is not returned.
    """
    condition = "nextPageToken" in response
    while condition:
        token = response["nextPageToken"]
        params_dict["pageToken"] = token
        response_obj = requests.get(base_playlist_url, params=params_dict)
        response = json.loads(response_obj.text)
        resp_items = response["items"]
        
        for item in resp_items:
            aList.append(item["snippet"]["resourceId"]["videoId"])
        condition = "nextPageToken" in response

    return aList

In [25]:
vid_ids = []

In [26]:
get_vid_ids(vid_ids, uid);

In [27]:
len(vid_ids)

1207

In [28]:
def get_vid_stats(aList, video_ids):
    """
    Returns a list of Lists with each SubList containing the Title, Description, Upload Date, Views, Likes,
    Dislikes, Comments and Watch Link for a Unique Video

    Required Arguements:
    aList:        A List where to store all the information about the video IDs gathered
    and to ultimately convert to a DataFrame
    video_ids:    A String (if only 1 Video ID) or a List contiaing Multiple Video IDs
    """

    """
    This while loop is created because at a given time, the API can only fetch data about
    a maximum of 50 videos. So, {x} starts at 0 and increments by 50 each time the loop is
    run. In every loop run, the data about the videos fetched is appended to a list and 
    the loop is run again to fetch the data for the next 50 videos in the {vid_ids} List.
    This keeps on going until x reaches a value >= len(vid_ids), at which point it stops.
    """
    x = 0
    while x < len(vid_ids):
        base_video_url = "https://www.googleapis.com/youtube/v3/videos"
        params_dict = {
        "part": "snippet, statistics",
        "id": vid_ids[x: x+50],    # Indexing is done to fetch data for 50 video IDs at a time.  
        "key": api_key
        }

        response_object = requests.get(base_video_url, params=params_dict)
        response = json.loads(response_object.text)
        resp_items = response["items"]

        for item in resp_items:
            title = item["snippet"]["title"]
            desc = item["snippet"]["description"]
            up_date = item["snippet"]["publishedAt"][:10]
            views = int(item["statistics"]["viewCount"])
            likes = int(item["statistics"]["likeCount"])
            dislikes = int(item["statistics"]["dislikeCount"])
            num_comments = int(item["statistics"]["commentCount"])
            vid_link = "https://www.youtube.com/watch?v={}".format(item["id"])

            aList.append([title, desc, up_date, views, likes, dislikes, num_comments, vid_link])
        x += 50

    return aList

In [29]:
df_list = []

In [30]:
get_vid_stats(df_list, vid_ids);

In [31]:
len(df_list)

1207

In [32]:
df_list[0]

['The NEAT Algorithm is Neat',
 'Code samples: https://github.com/Sentdex/NEAT-samples\nNeat-Python: https://neat-python.readthedocs.io/en/latest/\n\nChannel membership: https://www.youtube.com/channel/UCfzlCWGWYyIQ0aLC5w48gBQ/join\nDiscord: https://discord.gg/sentdex\nSupport the content: https://pythonprogramming.net/support-donate/\nTwitter: https://twitter.com/sentdex\nInstagram: https://instagram.com/sentdex\nFacebook: https://www.facebook.com/pythonprogramming.net/\nTwitch: https://www.twitch.tv/sentdex',
 '2021-01-07',
 4987,
 371,
 5,
 71,
 'https://www.youtube.com/watch?v=ZC0gMhYhwW0']

In [33]:
# Setting up Column names for our DataFrame
df_column_names = ["Title", "Description", "Uploaded On", "Views", "Likes", "Dislikes", "Comments", "Link"]

# Converting the Data gathered in our {df_list} List into a DataFrame
df = pd.DataFrame(data=df_list, columns=df_column_names)

## Basic Channel Statistics

In [66]:
print(f"Channel Title: {title}\n",
      f"Channel Description: {desc}\n\n",
      f"Total Subscribers: {subs}",
      sep="\n"
) 

Channel Title: sentdex

Channel Description: Python Programming tutorials, going further than just the basics. Learn about machine learning, finance, data analysis, robotics, web development, game development and more.

I have ~1000 videos, all of which do not fit on the front page. Try searching on the channel page, or via https://pythonprogramming.net. If you cannot find something, just ask! 

Bitcoin donations: 1GV7srgR4NJx4vrk7avCmmVQQrqmv87ty6
Paypal donations: HSKinsley@gmail.com


Total Subscribers: 996,000
Number of Videos uploaded: 1,207


#### A Brief Overview of All the videos uploaded on the Channel

In [34]:
df

,Title,Description,Uploaded On,Views,Likes,Dislikes,Comments,Link
0,The NEAT Algorithm is Neat,Code samples: https://github.com/Sentdex/NEAT-...,2021-01-07,4987,371,5,71,https://www.youtube.com/watch?v=ZC0gMhYhwW0
1,Neural Networks from Scratch - P.6 Softmax Act...,The what and why of the Softmax Activation fun...,2020-12-18,22534,1173,7,279,https://www.youtube.com/watch?v=omz_NdFgWyU
2,Neural Networks from Scratch (NNFS) in Print!,Get the book: https://nnfs.io\ntwitters: \ntwi...,2020-10-28,114549,5767,47,909,https://www.youtube.com/watch?v=sNHiM0DoEAg
3,Jetson Nano 2GB from NVIDIA,Get it here: https://www.nvidia.com/en-us/auto...,2020-10-14,31595,804,14,135,https://www.youtube.com/watch?v=iRrySp4VoPs
4,NNFS Update #2: Content done,More info: https://nnfs.io\n\nChannel membersh...,2020-09-25,26674,1330,14,291,https://www.youtube.com/watch?v=SN0VwEt2huo
...,...,...,...,...,...,...,...,...
1202,How to Sort a Python Dictionary By Value or Key!,Sentdex.com\nFacebook.com/sentdex\nTwitter.com...,2013-06-10,58415,392,27,34,https://www.youtube.com/watch?v=MGD_b2w_GU4
1203,Python's Logging Function,Sentdex.com\nFacebook.com/sentdex\nTwitter.com...,2013-06-08,24426,143,10,18,https://www.youtube.com/watch?v=OyZkXsgv5qk
1204,Python Encryption Tutorial with PyCrypto,Sentdex.com\nFacebook.com/sentdex\nTwitter.com...,2013-05-24,106696,556,31,123,https://www.youtube.com/watch?v=8PzDfykGg_g
1205,Matplotlib Python Tutorial Part 1: Basics and ...,Sample code: http://pythonprogramming.net/matp...,2013-05-16,170258,417,45,150,https://www.youtube.com/watch?v=wAwQ-noyB98


In [ ]:
avg_views = int(df["Views"].mean().round())
avg_likes = int(df["Likes"].mean().round())
avg_dislikes = int(df["Dislikes"].mean().round())
avg_comments = int(df["Comments"].mean().round())

print(f"Total views: {views}",
      f"Number of Videos uploaded: {vid_count}\n",
      f"Average Views per Video: {avg_views}",
      f"Average Likes per Video: {avg_likes}",
      f"Average Dislikes per Video: {avg_dislikes}",
      f"Average Comments per Video: {avg_comments}", sep="\n"
)

### Top 10 Most Viewed Videos

In [48]:
top_10_df = df.sort_values(by="Views", ascending=False).iloc[:10]
top_10_df

,Title,Description,Uploaded On,Views,Likes,Dislikes,Comments,Link
447,Practical Machine Learning Tutorial with Pytho...,The objective of this course is to give you a ...,2016-04-11,2253344,20459,247,971,https://www.youtube.com/watch?v=OGxgnH8y2NM
279,Self driving car neural network in the city - ...,"In this self-driving car with Python video, I ...",2017-04-21,1519127,21805,2482,1565,https://www.youtube.com/watch?v=KSX2psajYrg
446,Regression Intro - Practical Machine Learning ...,"To begin, what is regression in terms of us us...",2016-04-11,1202357,8836,185,1446,https://www.youtube.com/watch?v=JcI5Vnw0b2c
468,Introduction - Django Web Development with Pyt...,Welcome to a Django web development with Pytho...,2016-01-19,1060837,8839,180,767,https://www.youtube.com/watch?v=FNQxxpM1yOs
662,How to download and install Python Packages an...,This tutorial covers how to download and insta...,2015-01-21,1010406,4617,314,1166,https://www.youtube.com/watch?v=jnpC_Ib_lbc
757,Game Development in Python 3 With PyGame - 1 -...,"In this video, we introduce how to make video ...",2014-08-27,903786,8384,413,1178,https://www.youtube.com/watch?v=ujOTNg17LjI
152,"Deep Learning with Python, TensorFlow, and Ker...",An updated deep learning introduction using Py...,2018-08-11,831903,13635,229,1359,https://www.youtube.com/watch?v=wQ8BIBpya2k
378,What I do for a living - Q&A #1,"Sentdex Q&A. To start, I answer how I learned ...",2016-11-05,780181,12730,142,970,https://www.youtube.com/watch?v=eMtHmKO8GsA
494,Intro and loading Images - OpenCV with Python...,"Welcome to a tutorial series, covering OpenCV,...",2015-12-16,692341,4891,109,536,https://www.youtube.com/watch?v=Z78zbnLlPUA
344,Intro and Getting Stock Price Data - Python Pr...,Welcome to a Python for Finance tutorial serie...,2017-01-17,690071,7859,87,589,https://www.youtube.com/watch?v=2BrpKpWwT2A


In [49]:
top_avg_views = int(top_10_df["Views"].mean().round())
top_avg_likes = int(top_10_df["Likes"].mean().round())
top_avg_dislikes = int(top_10_df["Dislikes"].mean().round())
top_avg_comments = int(top_10_df["Comments"].mean().round())

print(f"Average Views per Video: {top_avg_views}",
f"Average Likes per Video: {top_avg_likes}",
f"Average Dislikes per Video: {top_avg_dislikes}",
f"Average Comments per Video: {top_avg_comments}",
sep="\n"
)

Average Views per Video: 1094435
Average Likes per Video: 11206
Average Dislikes per Video: 439
Average Comments per Video: 1055


### Top 10 Least Viewed VIdeos

In [50]:
bottom_10_df = df.sort_values(by="Views").iloc[:10]
bottom_10_df

,Title,Description,Uploaded On,Views,Likes,Dislikes,Comments,Link
1100,"The most ""open"" track day ever!",Quite possibly the best open track session I'v...,2013-09-16,164,5,1,0,https://www.youtube.com/watch?v=026FhTz6uIM
1084,Website structure - Search Engine Optimization...,Sentdex.com\nFacebook.com/sentdex\nTwitter.com...,2013-09-28,307,2,1,0,https://www.youtube.com/watch?v=sb2ramn0wow
1165,Basic PHP Programming Tutorial 25: Time and Ti...,Link to the full playlist: \nhttp://www.youtub...,2013-06-28,324,5,1,0,https://www.youtube.com/watch?v=S32NKR_fOoA
1164,Basic PHP Programming Tutorial 26: Die and Exit,Link to the full playlist: \nhttp://www.youtub...,2013-06-28,344,5,2,5,https://www.youtube.com/watch?v=RmPTbjsGbjI
1180,Basic PHP Tutorial 9: While Loop,Link to the full playlist: \nhttp://www.youtub...,2013-06-28,345,4,1,0,https://www.youtube.com/watch?v=PFdJc5MDxpA
1189,Basic PHP Tutorial 16: Include and Require,Link to the full playlist: \nhttp://www.youtub...,2013-06-28,370,4,1,0,https://www.youtube.com/watch?v=yDQnTsvS52w
1188,Basic PHP Tutorial 14: Foreach loop,Link to the full playlist: \nhttp://www.youtub...,2013-06-28,375,4,1,0,https://www.youtube.com/watch?v=urM5-7mvFVc
1181,Basic PHP Tutorial 10: Switch Statement,Link to the full playlist: \nhttp://www.youtub...,2013-06-28,398,2,2,0,https://www.youtube.com/watch?v=WvwD5chM9gg
1187,Basic PHP Tutorial 12: Associative Array,Link to the full playlist: \nhttp://www.youtub...,2013-06-28,411,3,1,2,https://www.youtube.com/watch?v=dxKRwuDwE9s
1174,"Basic PHP Tutorial 7: Assignment, comparison a...",Link to the full playlist: \nhttp://www.youtub...,2013-06-28,411,7,1,2,https://www.youtube.com/watch?v=6mIdkgO1u7Y


In [51]:
bottom_avg_views = int(bottom_10_df["Views"].mean().round())
bottom_avg_likes = int(bottom_10_df["Likes"].mean().round())
bottom_avg_dislikes = int(bottom_10_df["Dislikes"].mean().round())
bottom_avg_comments = int(bottom_10_df["Comments"].mean().round())

print(f"Average Views per Video: {bottom_avg_views}",
f"Average Likes per Video: {bottom_avg_likes}",
f"Average Dislikes per Video: {bottom_avg_dislikes}",
f"Average Comments per Video: {bottom_avg_comments}",
sep="\n"
)

Average Views per Video: 345
Average Likes per Video: 4
Average Dislikes per Video: 1
Average Comments per Video: 1
